In [1]:
import requests

page_id = "475799095608711"
access_token = "EAA2rrMSfqJsBPEwYZCCDJCK7G6cZAinU4ZA1nXOYyKm6Ly36OckMLbawch3TwtTGzapMfZACM4Qac2sH3soIuDT8hHpeLyiI4ittv3G1E0ZB4DBWORxR2EIgiESf3U2Icpujo1yf0FXW7TRmZBy1DG6fU5iQZCQtUDZBotrjbZBQJbEec09UY9wBYIpllnPYg6WfJUVZAD8gqjChsM9WxCbyqtFHDLNlXwpkxPjWkQTy0n"
url = f"https://graph.facebook.com/{page_id}/posts?fields=message&access_token={access_token}"

response = requests.get(url)

if response.status_code == 200:
    response_json = response.json()
    captions = [post["message"] for post in response_json.get("data", []) if "message" in post]
    print("Extracted Captions:")
    for caption in captions:
        print(caption)
else:
    print(f"Error: {response.status_code}, {response.text}")


Extracted Captions:
DKTE’s Textile Students Selected by Reed & Tailor Company
The news of DKTE’s Textile students being successfully placed at Reed & Tailor Company has been widely published by various leading newspapers.
...........................................................................................
DKTE Society's
TEXTILE AND ENGINEERING INSTITUTE, ICHALKARANJI
AN EMPOWERED AUTONOMOUS INSTITUTE
NAAC ACCREDITED WITH 'A+' GRADE
...........................................................................................
#DKTE #CampusPlacement #SuccessStory #TextileIndustry
A one-week FDP on Testing of Technical Textiles was organized from 2nd to 7th December 2024 by Department of Textiles and AICTE Idea lab in Collaboration with Institutions Innovation Council.
The FDP included prominent speakers, including Ms. Seema Patel, Joint Director, Testing and Analysis, Wool Research Association, Mumbai. Ms. Seema's enlightening lesson focused on "Evaluation of Protective Textiles, Geo

# New Section

In [2]:
import csv

# Save captions to a CSV file
with open("captions.csv", "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Caption"])  # Write header
    for caption in captions:
        writer.writerow([caption])

print("Captions saved to captions.csv")

Captions saved to captions.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

In [5]:
# Step 1: Load the training dataset
train_file_path = '/content/training_data_set.xlsx'
train_data = pd.read_excel(train_file_path)
# Rename columns for clarity
train_data.columns = ['SerialNo', 'Post', 'Label']



In [6]:
# Step 2: Preprocess the training data
train_data.dropna(subset=['Post', 'Label'], inplace=True)
X_train = train_data['Post']
y_train = train_data['Label']

In [7]:
# Step 3: Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)

In [8]:
# Step 4: Train a classification model
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train_tfidf, y_train)

# Save the trained model and vectorizer
joblib.dump(model, 'post_classification_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Model trained and saved successfully.")

Model trained and saved successfully.


In [9]:
# Step 5: Load the captions dataset for prediction
captions_file_path = '/content/captions.csv'
captions_data = pd.read_csv(captions_file_path)

# Ensure the column name is consistent
captions_data.columns = ['Post']

In [10]:
# Step 6: Preprocess the captions data
captions_data.dropna(subset=['Post'], inplace=True)
X_captions = captions_data['Post']

# Transform the captions using the saved vectorizer
X_captions_tfidf = vectorizer.transform(X_captions)


In [11]:
# Step 7: Predict the classes
predicted_labels = model.predict(X_captions_tfidf)


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X = raw captions
X = train_data['Post']   # list of text captions
y = train_data['Label']  # list of their actual class labels

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Predict on test set
predicted_labels = model.predict(X_test_tfidf)

# Accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 96.77%


In [23]:

import os

In [13]:
# Step 8: Save the predictions to a new file
captions_data['Predicted_Label'] = predicted_labels
output_file_path = '/mnt/data/captions_with_predictions.csv'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

captions_data.to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}.")


Predictions saved to /mnt/data/captions_with_predictions.csv.


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
import pandas as pd
import re
import spacy

# Load the English NLP model from spaCy
nlp = spacy.load("en_core_web_sm")

# Define a function to extract structured information
def extract_information_with_participant_names(caption):
    # Process the caption using spaCy
    doc = nlp(caption)

    # Extract a heading
    heading = caption.split(":")[0] if ":" in caption else caption[:50]

    # Regex patterns for date, duration, and department
    date_pattern = r"\b(?:\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4}|\w+\s+\d{1,2},?\s+\d{4})\b"
    duration_pattern = r"\b(?:\d+\s+(?:hour|day|week|month|year)s?)\b"
    department_pattern = r"\b(?:Department of \w+|Electrical Engineering|Mechanical Engineering|Computer Science)\b"

    # Extract entities using regex
    date = re.search(date_pattern, caption)
    duration = re.search(duration_pattern, caption)
    department = re.search(department_pattern, caption, re.IGNORECASE)

    # Extract participant names using spaCy
    participants = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

    # Return structured information
    return {
        "Heading": heading.strip(),
        "Date": date.group(0) if date else None,
        "Duration": duration.group(0) if duration else None,
        "Department": department.group(0) if department else None,
        "Participants": ", ".join(participants) if participants else None,
    }

# Step 1: Load the captions dataset with predictions
captions_file_path = '/mnt/data/captions_with_predictions.csv'
captions_data = pd.read_csv(captions_file_path)

# Ensure the column names are consistent
captions_data.columns = ['Post', 'Predicted_Label']

# Step 2: Apply the NLP extraction function
structured_data = captions_data['Post'].apply(extract_information_with_participant_names)
structured_df = pd.DataFrame(list(structured_data))

# Combine the structured information with the original data
result_df = pd.concat([captions_data, structured_df], axis=1)

# Step 3: Save the combined structured data to a single file
output_file_path = '/mnt/data/structured_captions_with_classification.csv'
result_df.to_csv(output_file_path, index=False, encoding="utf-8")

print(f"Structured data with classification has been saved to {output_file_path}.")


Structured data with classification has been saved to /mnt/data/structured_captions_with_classification.csv.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>